In [190]:
import cv2
import numpy as np
from tqdm import tqdm
import os

REBUILD_DATA = True
CAT = 'datasets/dog_and_cat/training_set/training_set/cats'
DOG = 'datasets/dog_and_cat/training_set/training_set/dogs'
CAT_TEST = 'datasets/dog_and_cat/test_set/test_set/cats'
DOG_TEST = 'datasets/dog_and_cat/test_set/test_set/dogs'
SAVE_FILE = 'datasets/dog_and_cat/data_cat_dog_processed.npy'
LABELS = {CAT: 0, CAT_TEST: 0, DOG: 1, DOG_TEST: 1}
IMG_SIZE = 64

class DogvsCat():

  def __init__(self):
    self.data = []
    self.cat_count = 0
    self.dog_count = 0

  def make_training_data(self):
    for label in LABELS:
      print(label)
      for f in tqdm(os.listdir(label)):
        if 'jpg' in f:
          path = os.path.join(label, f)
          img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
          img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
          self.data.append([np.array(img), LABELS[label]])
          if label == CAT:
            self.cat_count += 1
          if label == DOG:
            self.dog_count += 1

    np.random.shuffle(self.data)
    np.save(SAVE_FILE, self.data)
    print(f'cat count: {self.cat_count}, dog count: {self.dog_count}')

if REBUILD_DATA:
  dogvscat = DogvsCat()
  dogvscat.make_training_data()

datasets/dog_and_cat/training_set/training_set/cats


  0%|          | 0/4001 [00:00<?, ?it/s]

100%|██████████| 4001/4001 [00:02<00:00, 1597.04it/s]


datasets/dog_and_cat/test_set/test_set/cats


100%|██████████| 1012/1012 [00:00<00:00, 1581.34it/s]


datasets/dog_and_cat/training_set/training_set/dogs


100%|██████████| 4006/4006 [00:02<00:00, 1493.23it/s]


datasets/dog_and_cat/test_set/test_set/dogs


100%|██████████| 1013/1013 [00:00<00:00, 1499.23it/s]


cat count: 4000, dog count: 4005


In [121]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.


In [191]:
from sklearn.model_selection import train_test_split

data = np.load(SAVE_FILE, allow_pickle=True)
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

X_train = torch.Tensor([i[0] for i in train]).view(-1, 1, IMG_SIZE, IMG_SIZE)
y_train = torch.Tensor([i[1] for i in train])
X_train, y_train = X_train.to(device), y_train.to(device)

X_test = torch.Tensor([i[0] for i in test]).view(-1, 1, IMG_SIZE, IMG_SIZE)
y_test = torch.Tensor([i[1] for i in test])
X_test, y_test = X_test.to(device), y_test.to(device)

X_train, X_test = X_train/255.0, X_test/255.0

train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

test_data = torch.utils.data.TensorDataset(X_test, y_test)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


torch.Size([8022, 1, 64, 64]) torch.Size([8022]) torch.Size([2006, 1, 64, 64]) torch.Size([2006])


In [187]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=4, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=1)

        self.fc1 = nn.Linear(128*15*15, 128)
        self.fc2 = nn.Linear(128, 256)

    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 128*15*15)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)

        return x



In [188]:

EPOCHS = 20

net = Net()
net = net.to(device)
print(net)
net.zero_grad()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(EPOCHS):
  for data in tqdm(train_data_loader):
    X, y = data
    y = y.long()
    net.zero_grad()
    output = net(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
  print(f'Epoch: {epoch}, loss: {loss}')

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=4608, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=256, bias=True)
)


  0%|          | 0/251 [00:00<?, ?it/s]

100%|██████████| 251/251 [00:00<00:00, 253.66it/s]


Epoch: 0, loss: 0.6189684867858887


100%|██████████| 251/251 [00:00<00:00, 295.17it/s]


...

Epoch: 17, loss: 0.010456277057528496


100%|██████████| 251/251 [00:00<00:00, 294.64it/s]


Epoch: 18, loss: 0.16468463838100433


100%|██████████| 251/251 [00:00<00:00, 292.99it/s]

Epoch: 19, loss: 0.027447042986750603


In [189]:
with torch.no_grad():
  correct = 0
  total = 0
  for data in test_data_loader:
    X, y = data
    y = y.long()
    output = net(X)
    for idx, i in enumerate(output):
      if torch.argmax(i) == y[idx]:
        correct += 1
      total += 1
  print(f'Accuracy: {round(correct/total, 3)}')

Accuracy: 0.764
